In [35]:
pip install scipy --quiet

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install tenacity --quiet

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install tiktoken --quiet

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install termcolor --quiet

Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install requests --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import requests

GPT_MODEL = "gpt-4o"
client = OpenAI(api_key="api-key")

In [42]:
#First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [43]:
#Let's create some function specifications to interface with a weather API. 
#We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]


# Define the OpenWeatherMap API key and endpoint
#API_KEY = "your_openweathermap_api_key"
API_KEY = "2f7e826e11ea5f45ec370369c683c670"
CURRENT_WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
FORECAST_WEATHER_URL = "https://api.openweathermap.org/data/2.5/forecast"

#Now let's implement the functions that will actually fetch the real-time data from the OpenWeatherMap API.

# Function to get current weather
def get_current_weather(location, format="celsius"):
    units = "metric" if format == "celsius" else "imperial"
    params = {
        "q": location,
        "appid": API_KEY,
        "units": units
    }
    response = requests.get(CURRENT_WEATHER_URL, params=params)
    
    if response.status_code == 200:
        return response.json()  # The data fetched from OpenWeatherMap
    else:
        return {"error": "Failed to fetch current weather data"}

# Function to get N-day weather forecast
def get_n_day_weather_forecast(location, num_days, format="celsius"):
    units = "metric" if format == "celsius" else "imperial"
    params = {
        "q": location,
        "cnt": num_days,  # Number of days in forecast
        "appid": API_KEY,
        "units": units
    }
    response = requests.get(FORECAST_WEATHER_URL, params=params)
    
    if response.status_code == 200:
        return response.json()  # The data fetched from OpenWeatherMap
    else:
        return {"error": "Failed to fetch weather forecast"}


In [44]:
#If we prompt the model about the current weather, it will respond with some clarifying questions.

messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})

messages.append({"role": "user", "content": "What will the weather be like for the next three days?"})

In [45]:
# 수정된 run_conversation_enhanced_v2 함수 - 사용자 질문이 로그에 표시됨
def run_conversation_enhanced():
    """수정된 LLM과 사용자 간의 대화를 처리하는 메인 함수 - 사용자 질문 로그 표시"""

    # 시스템 메시지 설정
    messages = [
        {
            "role": "system",
            "content": """
            당신은 날씨 정보를 제공하는 AI 어시스턴트입니다. 

            중요한 규칙:
            1. 한국어 도시명을 영어로 변환하여 함수에 전달하세요:
            - 서울 → Seoul
            - 부산 → Busan  
            - 대구 → Daegu
            - 인천 → Incheon
            - 광주 → Gwangju
            - 대전 → Daejeon
            - 울산 → Ulsan
            - 도쿄 → Tokyo
            - 오사카 → Osaka
            - 베이징 → Beijing
            - 상하이 → Shanghai
            - 타이베이 → Taipei
            - 기타 한국어/일본어/중국어 도시명도 영어로 변환

            2. 사용자가 특정 도시의 현재 날씨나 향후 며칠간의 날씨 예측을 요청하면, 적절한 함수를 사용하여 OpenWeatherMap API에서 실시간 데이터를 가져와 응답하세요.

            3. 사용자의 요청이 모호할 경우 명확히 질문하세요. 예를 들어:
                - "날씨 알려줘" → "어느 도시의 날씨를 원하시나요?"
                - "3일간 날씨" → "어느 도시의 3일간 날씨 예측을 원하시나요?"

            함수에서 받은 날씨 데이터를 사용자에게 친화적이고 이해하기 쉽게 설명해주세요.
            """,
        }
    ]

    print("=== 날씨 정보 AI 어시스턴트 ===")
    print("현재 날씨나 향후 며칠간의 날씨 예측을 요청하세요.")
    print("예시:")
    print("  - '서울의 현재 날씨 알려줘'")
    print("  - '뉴욕의 향후 5일간 날씨 예측해줘'")
    print("  - '도쿄 3일간 날씨'")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.\n")

    while True:
        # 사용자 입력 받기
        user_input = input("사용자: ").strip()

        if user_input.lower() in ['quit', 'exit', '종료']:
            print("🤖 어시스턴트: 대화를 종료합니다. 좋은 하루 되세요!")
            break

        if not user_input:
            continue

        # 사용자 입력을 로그에 명시적으로 표시 (중요!)
        print(f"👤 사용자: {user_input}")

        # 사용자 메시지 추가
        messages.append({"role": "user", "content": user_input})

        # LLM 응답 생성
        response = chat_completion_request(messages, tools=tools)

        if isinstance(response, Exception):
            print(f"🤖 어시스턴트: 죄송합니다. 오류가 발생했습니다: {response}")
            continue

        assistant_message = response.choices[0].message
        messages.append(assistant_message)

        # 함수 호출이 필요한 경우
        if assistant_message.tool_calls:
            # 어시스턴트의 초기 응답이 있는 경우에만 출력
            if assistant_message.content:
                print(f"🤖 어시스턴트: {assistant_message.content}")

            # 각 함수 호출 처리
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                # 함수 호출 정보를 더 자연스럽게 표시
                if function_name == "get_current_weather":
                    location = function_args.get("location")
                    print(f"🔍 {location}의 현재 날씨를 조회하고 있습니다...")
                elif function_name == "get_n_day_weather_forecast":
                    location = function_args.get("location")
                    num_days = function_args.get("num_days")
                    print(f"🔍 {location}의 {num_days}일간 날씨 예보를 조회하고 있습니다...")

                # 함수 실행
                if function_name == "get_current_weather":
                    function_response = get_current_weather(
                        location=function_args.get("location"),
                        format=function_args.get("format")
                    )
                elif function_name == "get_n_day_weather_forecast":
                    function_response = get_n_day_weather_forecast(
                        location=function_args.get("location"),
                        num_days=function_args.get("num_days"),
                        format=function_args.get("format")
                    )
                else:
                    function_response = {"error": f"알 수 없는 함수: {function_name}"}

                # 함수 결과를 메시지에 추가 (tool_call_id 포함)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(function_response, ensure_ascii=False)
                })

            # 함수 결과를 바탕으로 최종 응답 생성
            final_response = chat_completion_request(messages, tools=tools)

            if not isinstance(final_response, Exception):
                final_message = final_response.choices[0].message
                messages.append(final_message)
                print(f"🤖 어시스턴트: {final_message.content}")
        else:
            # 함수 호출이 없는 경우 직접 응답
            print(f"🤖 어시스턴트: {assistant_message.content}")

        print()  # 빈 줄 추가

In [46]:
# 날씨 데이터를 사용자 친화적으로 포맷하는 함수들
def format_current_weather(weather_data):
    """현재 날씨 데이터를 사용자 친화적으로 포맷"""
    if "error" in weather_data:
        return f"오류: {weather_data['error']}"

    try:
        city = weather_data["name"]
        country = weather_data["sys"]["country"]
        temp = weather_data["main"]["temp"]
        feels_like = weather_data["main"]["feels_like"]
        humidity = weather_data["main"]["humidity"]
        description = weather_data["weather"][0]["description"]
        wind_speed = weather_data["wind"]["speed"]

        return f"""
        📍 {city}, {country}\n🌡️ 온도: {temp}°C (체감온도: {feels_like}°C)\n☁️ 날씨: {description}\n💧 습도: {humidity}%\n💨 풍속: {wind_speed} m/s
        """.strip()
    except KeyError as e:
        return f"데이터 파싱 오류: {e}"


def format_forecast_weather(forecast_data):
    """날씨 예측 데이터를 사용자 친화적으로 포맷"""
    if "error" in forecast_data:
        return f"오류: {forecast_data['error']}"

    try:
        city = forecast_data["city"]["name"]
        country = forecast_data["city"]["country"]
        forecasts = forecast_data["list"]

        result = f"📍 {city}, {country} - {len(forecasts)}일간 날씨 예측\n\n"

        for i, forecast in enumerate(forecasts[:5]):  # 최대 5일까지만 표시
            date = forecast["dt_txt"]
            temp = forecast["main"]["temp"]
            description = forecast["weather"][0]["description"]
            humidity = forecast["main"]["humidity"]

            result += f"📅 {date}\n"
            result += f"🌡️ 온도: {temp}°C\n"
            result += f"☁️ 날씨: {description}\n"
            result += f"💧 습도: {humidity}%\n\n"

        return result.strip()
    except KeyError as e:
        return f"데이터 파싱 오류: {e}"

In [47]:
# 서울 현재 날씨 테스트
result = get_current_weather("Seoul", "celsius")
print("서울 현재 날씨:")
print(format_current_weather(result))
print()

# 서울 5일간 날씨 예측 테스트
result = get_n_day_weather_forecast("Seoul", 5, "celsius")
print("서울 5일간 날씨 예측:")
print(format_forecast_weather(result))
print()


서울 현재 날씨:
📍 Seoul, KR
🌡️ 온도: 21.4°C (체감온도: 21.42°C)
☁️ 날씨: clear sky
💧 습도: 70%
💨 풍속: 1.73 m/s

서울 5일간 날씨 예측:
📍 Seoul, KR - 5일간 날씨 예측

📅 2025-10-08 15:00:00
🌡️ 온도: 21.32°C
☁️ 날씨: clear sky
💧 습도: 72%

📅 2025-10-08 18:00:00
🌡️ 온도: 20.29°C
☁️ 날씨: few clouds
💧 습도: 70%

📅 2025-10-08 21:00:00
🌡️ 온도: 18.85°C
☁️ 날씨: overcast clouds
💧 습도: 67%

📅 2025-10-09 00:00:00
🌡️ 온도: 21.26°C
☁️ 날씨: overcast clouds
💧 습도: 63%

📅 2025-10-09 03:00:00
🌡️ 온도: 23.1°C
☁️ 날씨: broken clouds
💧 습도: 54%



In [48]:
# 도쿄 3일간 날씨 예측 테스트
result = get_n_day_weather_forecast("Tokyo", 3, "celsius")
print("도쿄 3일간 날씨 예측:")
print(result['list'][0])
print(format_forecast_weather(result))

도쿄 3일간 날씨 예측:
{'dt': 1759935600, 'main': {'temp': 24.64, 'feels_like': 24.8, 'temp_min': 24.15, 'temp_max': 24.64, 'pressure': 1014, 'sea_level': 1014, 'grnd_level': 1012, 'humidity': 63, 'temp_kf': 0.49}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.39, 'deg': 58, 'gust': 5.58}, 'visibility': 10000, 'pop': 0.43, 'rain': {'3h': 0.31}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-10-08 15:00:00'}
📍 Tokyo, JP - 3일간 날씨 예측

📅 2025-10-08 15:00:00
🌡️ 온도: 24.64°C
☁️ 날씨: light rain
💧 습도: 63%

📅 2025-10-08 18:00:00
🌡️ 온도: 23.55°C
☁️ 날씨: overcast clouds
💧 습도: 70%

📅 2025-10-08 21:00:00
🌡️ 온도: 21.92°C
☁️ 날씨: overcast clouds
💧 습도: 71%


In [49]:
run_conversation_enhanced()

=== 날씨 정보 AI 어시스턴트 ===
현재 날씨나 향후 며칠간의 날씨 예측을 요청하세요.
예시:
  - '서울의 현재 날씨 알려줘'
  - '뉴욕의 향후 5일간 날씨 예측해줘'
  - '도쿄 3일간 날씨'
종료하려면 'quit' 또는 'exit'를 입력하세요.

👤 사용자: 서울의 현재 날씨를 알려줘
🔍 Seoul의 현재 날씨를 조회하고 있습니다...
🤖 어시스턴트: 서울의 현재 날씨는 맑은 하늘이며, 기온은 약 21.4°C입니다. 체감 온도도 거의 동일하게 느껴집니다. 바람은 약간 불고 있으며, 풍속은 1.73m/s입니다. 습도는 70%로 다소 높습니다. 하늘은 거의 구름이 없는 상태로, 쾌청한 날씨를 즐길 수 있습니다.

👤 사용자: 서울의 3일간 날씨를 알려줘
🔍 Seoul의 3일간 날씨 예보를 조회하고 있습니다...
🤖 어시스턴트: 서울의 향후 3일간 날씨 예보는 다음과 같습니다:

1. **첫째 날 (2025-10-08)**
   - **날씨:** 맑은 하늘
   - **기온:** 약 21.32°C, 체감 온도 21.39°C
   - **습도:** 72%
   - **바람:** 북동풍, 속도 1.35 m/s

2. **둘째 날 (2025-10-08)**
   - **날씨:** 구름 조금
   - **기온:** 약 20.29°C, 체감 온도 20.2°C
   - **습도:** 70%
   - **바람:** 북동풍, 속도 2.21 m/s

3. **셋째 날 (2025-10-08)**
   - **날씨:** 흐림
   - **기온:** 약 18.85°C, 체감 온도 18.54°C
   - **습도:** 67%
   - **바람:** 북동풍, 속도 3.24 m/s

전반적으로 맑은 날씨가 지속되나 약간의 구름이 낄 수 있으며, 마지막 날은 흐릴 예정입니다. 바람은 약간 있지만 크게 강하지 않아 쾌적한 날씨를 기대할 수 있습니다.

👤 사용자: 도쿄의 내일 날씨를 알려줘
🔍 Tokyo의 1일간 날씨 예보를 조회하고 있습니다...
🤖 어시스턴트: 도쿄의 내일